# Part 3: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

Authors:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)

In [1]:
import syft as sy
hook = sy.TorchHook()

# Section 3.1 - Pointers to Pointers

As you know, PointerTensor objects feel just like normal tensors. In fact, they are _so much like tensors_ that we can even have pointers TO the pointers. Check it out!

In [2]:
bob = sy.VirtualWorker(id='bob')
alice = sy.VirtualWorker(id='alice')

# making sure that bob/alice know about each other
bob.add_worker(alice)
alice.add_worker(bob)

In [3]:
# this is a local tensor
x = sy.FloatTensor([1,2,3,4])
x


 1
 2
 3
 4
[syft.core.frameworks.torch.tensor.FloatTensor of size 4]

In [4]:
# this sends the local tensor to Bob
x_ptr = x.send(bob)

# this is now a pointer
x_ptr

FloatTensor[_PointerTensor - id:24218495186 owner:me loc:bob id@loc:67334353600]

In [5]:
x_ptr.child.original_pointer

True

In [6]:
# now we can SEND THE POINTER to alice!!!
pointer_to_x_ptr = x_ptr.send(alice)

pointer_to_x_ptr

FloatTensor[_PointerTensor - id:24218495186 owner:me loc:alice id@loc:81646687778]

### What happened?

So, in the previous example, we created a tensor called "x" and send it to Bob, creating a pointer on our local machine ("x_ptr"). 

Then, we called x_ptr.send(alice) which SENT THE POINTER to Alice. 

Note, this did NOT move the data! Instead, it moved the pointer to the data!! 

In [7]:
# As you can see above, Bob still has the actual data (data is always stored in a LocalTensor type). 
bob._objects

{67334353600: [_LocalTensor - id:67334353600 owner:bob]}

In [8]:
# Alice, on the other hand, has x_ptr!! (notice how it points at bob)
alice._objects

{81646687778: [_PointerTensor - id:81646687778 owner:alice loc:bob id@loc:67334353600]}

In [9]:
# and we can use .get() to get x_ptr back from Alice

x_ptr = pointer_to_x_ptr.get()
x_ptr

FloatTensor[_PointerTensor - id:24218495186 owner:me loc:bob id@loc:67334353600]

In [10]:
# and then we can use x_ptr to get x back from Bob!

x = x_ptr.get()
x


 1
 2
 3
 4
[syft.core.frameworks.torch.tensor.FloatTensor of size 4]

### Arithmetic on Pointer -> Pointer -> Data Object

And just like with normal pointers, we can perform arbitrary PyTorch operations across these tensors

In [11]:
bob._objects

{}

In [12]:
alice._objects

{}

In [13]:
p2p2x = sy.FloatTensor([1,2,3,4,5]).send(bob).send(alice)

y = p2p2x + p2p2x

In [14]:
bob._objects

{59135289588: [_LocalTensor - id:59135289588 owner:bob],
 85001897033: [_LocalTensor - id:85001897033 owner:bob]}

In [15]:
alice._objects

{20277136977: [_PointerTensor - id:20277136977 owner:alice loc:bob id@loc:59135289588],
 45786039766: [_PointerTensor - id:45786039766 owner:alice loc:bob id@loc:85001897033]}

In [16]:
y.get().get()


  2
  4
  6
  8
 10
[syft.core.frameworks.torch.tensor.FloatTensor of size 5]

In [17]:
bob._objects

{59135289588: [_LocalTensor - id:59135289588 owner:bob]}

In [18]:
alice._objects

{20277136977: [_PointerTensor - id:20277136977 owner:alice loc:bob id@loc:59135289588]}

In [19]:
p2p2x.get().get()


 1
 2
 3
 4
 5
[syft.core.frameworks.torch.tensor.FloatTensor of size 5]

In [20]:
bob._objects

{}

In [21]:
alice._objects

{}

# Section 3.2 - Pointer Chain Operations

So in the last section whenever we called a .send() or a .get() operation, it called that operation directly on the tensor on our local machine. However, if you have a chain of pointers, sometimes you want to call operations like .get() or .send() on the LAST pointer in the chain (such as sending data directly from one worker to another). To accomplish this, you want to use functions which are especially designed for this privacy preserving operation.

These operations are:

- my_poitner2pointer.end_get()
- my_pointer2pointer.move(another_worker)

Let's start with .end_get(). This one simply identifies the _last_ pointer in the chain and calls .get() on that pointer! It's an inline operation. Let's look at an example.

In [22]:
# x is now a pointer to a pointer to the data which lives on Bob's machine
x = sy.FloatTensor([1,2,3,4,5]).send(bob).send(alice)

In [23]:
bob._objects

{5053492161: [_LocalTensor - id:5053492161 owner:bob]}

In [24]:
alice._objects

{37740619399: [_PointerTensor - id:37740619399 owner:alice loc:bob id@loc:5053492161]}

In [26]:
x2 = x.end_get()

In [27]:
x2

FloatTensor[_PointerTensor - id:95839402744 owner:me loc:alice id@loc:37740619399]

In [28]:
bob._objects

{}

In [29]:
alice._objects

{37740619399: [_LocalTensor - id:37740619399 owner:alice]}

In [30]:
x2

FloatTensor[_PointerTensor - id:95839402744 owner:me loc:alice id@loc:37740619399]

In [31]:
x2.get()


 1
 2
 3
 4
 5
[syft.core.frameworks.torch.tensor.FloatTensor of size 5]

### Analyzing .end_get()

Notice above when we called .end_get(), it deleted bob's object and MOVED it to alice. So now Alice has the actual data (a LocalTensor). Thus, when we now call .get() on "x2" we will get the data back.

Now, you'll notice, before we called x2.get() we actually sent our tensor on a little journey. 

- First we sent the data to Bob. 
- Then we sent a pointer to the data to Alice
- Then we used Alice's pointer to MOVE the data to Alice (by calling .end_get())

Thus, we used this series of operations to MOVE the data from Bob -> Alice without us actually seeing the data during the in-between step. As you might guess - .move() is just a convenience wrapper around this operation!

In [32]:
# x is now a pointer to a pointer to the data which lives on Bob's machine
x = sy.Variable(sy.FloatTensor([1,2,3,4,5])).send(bob)

In [34]:
bob._objects

{7336259882: [_LocalTensor - id:7336259882 owner:bob],
 10911662817: [_LocalTensor - id:10911662817 owner:bob],
 23961023424: [_LocalTensor - id:23961023424 owner:bob],
 90522227459: [_LocalTensor - id:90522227459 owner:bob]}

In [35]:
alice._objects

{}

In [36]:
x = x.move(alice)

In [37]:
bob._objects

{}

In [38]:
alice._objects

{2787261370: [_LocalTensor - id:2787261370 owner:alice],
 23961023424: [_LocalTensor - id:23961023424 owner:alice],
 54870725261: [_LocalTensor - id:54870725261 owner:alice],
 81612366583: [_PointerTensor - id:81612366583 owner:alice loc:bob id@loc:23961023424],
 87049828237: [_PointerTensor - id:87049828237 owner:alice loc:bob id@loc:90522227459],
 90522227459: [_LocalTensor - id:90522227459 owner:alice]}

In [39]:
x

Variable containing:FloatTensor[_PointerTensor - id:84116265428 owner:me loc:alice id@loc:54870725261]

# Congratulations!!! - Time to Join the Community!

Congraulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways!

### Star PySyft on Github

The easiest way to help our community is just by starring the Repos! This helps raise awareness of the cool tools we're building.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Join our Slack!

The best way to keep up to date on the latest advancements is to join our community! You can do so by filling out the form at [http://slack.openmined.org](http://slack.openmined.org)

### Join a Code Project!

The best way to contribute to our community is to become a code contributor! At any time you can go to PySyft Github Issues page and filter for "Projects". This will show you all the top level Tickets giving an overview of what projects you can join! If you don't want to join a project, but you would like to do a bit of coding, you can also look for more "one off" mini-projects by searching for github issues marked "good first issue".

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)